## Загружаем библиотеки, указываем с помощью какой модели будем делать прогноз

In [1]:
import os
import gc
import glob
import sys
import time
import random
from datetime import datetime
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, KFold

import torch
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import torch.nn as nn
from torch.utils.data import Dataset, Subset

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_metric_learning.losses import ArcFaceLoss

TIMMMODEL = 'tf_efficientnetv2_m_in21ft1k'
VERSION = '1207'
EMB_FEATURES = 256

SEED = 0
SIZE = 384
DEVICE = 'cuda'

def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

seed_everything(SEED)

## При формировании головы изменяем forward, т.к. нам требуется только регрессионный прогноз

In [2]:
class Head(nn.Module):
    def __init__(self, in_features, emb_features:int=512):
        super().__init__()

        self.in_features = in_features
        self.emb_features = emb_features
        
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Sequential(
            nn.Linear(self.in_features, self.emb_features, bias=False),
            nn.BatchNorm1d(self.emb_features),
            nn.ReLU(inplace=True),
            nn.Linear(self.emb_features, 1)
        )
        self.neck = nn.Sequential(
            nn.BatchNorm1d(self.in_features),
            nn.Linear(self.in_features, self.emb_features, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(self.emb_features),
            nn.Linear(self.emb_features, self.emb_features, bias=False),
            nn.BatchNorm1d(self.emb_features)
        )
        
        
    def forward(self, features):
        x = self.dropout(features)
        x = self.out(x)
        
        return x


model = timm.create_model(TIMMMODEL, pretrained=False)
head_name = model.default_cfg['classifier']

if head_name == 'last_linear':
    model.last_linear = Head(model.last_linear.in_features, emb_features=EMB_FEATURES)
elif head_name == 'head':
    model.head = Head(model.head.in_features, emb_features=EMB_FEATURES)
elif head_name == 'fc':
    model.fc = Head(model.fc.in_features, emb_features=EMB_FEATURES)
elif head_name == 'head.fc':
    model.head.fc = Head(model.head.fc.in_features, emb_features=EMB_FEATURES)
elif head_name == 'classifier':
    model.classifier = Head(model.classifier.in_features, emb_features=EMB_FEATURES)

model.load_state_dict(torch.load(f"{VERSION}_{TIMMMODEL}_arc.pth", map_location='cpu'))

model.eval()
_ = model.to(DEVICE)

## Определяем датасет

In [3]:
class ImageDataset(Dataset):
    def __init__(self, data_df, transform=None):

        self.data_df = data_df
        self.transform = transform

    def __getitem__(self, idx):
        # достаем имя изображения и ее лейбл
        image_name = self.data_df.iloc[idx]['img_num']

        # читаем картинку. read the image
        image = cv2.imread(f"test/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image=image)['image']
        
        return image
    
    def __len__(self):
        return len(self.data_df)


transform_test = A.Compose([
    A.Resize(SIZE, SIZE),
    A.Normalize(mean= model.default_cfg['mean'],
                std = model.default_cfg['std']),
    ToTensorV2()
])

test_df = pd.read_csv("sample_solution.csv")
test_df = test_df.drop(["number_of_houses"], axis = 1)

dataset = ImageDataset(test_df, transform_test)

dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=4,
                                         shuffle=False,
                                         pin_memory=True,
                                         drop_last=False,
                                         num_workers=4)

## Делаем предсказание

In [4]:
results = []
with torch.no_grad():
    for it, imgs in enumerate(dataloader, start = 1):
        if it % 10 == 0:
            print(f'{datetime.now().strftime("%H:%M:%S")} iter {it:5d}')
        
        imgs = imgs.to(DEVICE)
        y_pred = model(imgs)
        
        pred = torch.clip(y_pred, 1.0, 32.0)
        pred = torch.ceil(pred)

        pred_numpy = pred.cpu().numpy().astype('int').flatten()

        results.extend(pred_numpy.tolist())

16:54:41 iter    10
16:54:42 iter    20
16:54:43 iter    30
16:54:44 iter    40
16:54:45 iter    50
16:54:46 iter    60
16:54:47 iter    70
16:54:48 iter    80
16:54:50 iter    90
16:54:51 iter   100
16:54:52 iter   110
16:54:53 iter   120
16:54:54 iter   130
16:54:55 iter   140
16:54:57 iter   150
16:54:58 iter   160
16:54:59 iter   170
16:55:00 iter   180
16:55:01 iter   190
16:55:02 iter   200
16:55:03 iter   210
16:55:04 iter   220


## Сохраняем результат

In [5]:
test_df["number_of_houses"] = results

test_df.to_csv(f"{VERSION}_{TIMMMODEL}_arc.csv", index=False)
test_df.head(3)

,img_num,number_of_houses
0,000000001801.jpg,6
1,000000001190.jpg,2
2,000000001817.jpg,16
